In [1]:
import BhrLgcGPTProcess
import BhrLgcManualProcess

import pandas as pd
import numpy as np
import json
import re
import pickle
import hashlib

Current working directory: /Users/jackhan/Desktop/AITown/satoshiLive/satoshiLiveAI/BhrCtrl
Config sections found: ['mysql', 'Paths', 'OpenAI', 'NetworkSocket']
YAML content loaded successfully.


In [2]:
java_json = """

{"world": {"time": 1878522183069}, "npcs": [{"npcId": 10006, "status": "free", "selling": [], "items": [], "action": {"param": {"gridX": 0.0, "gridY": 0.0, "npcId": 10010.0}}, "curAction": {}, "mapData": [], "surroundings": {"people": [{"npcId": 10007, "status": "free"}, {"npcId": 10008, "status": "free"}, {"npcId": 10009, "status": "free"}, {"npcId": 10010, "status": "move"}], "items": []}, "talk": {"isTalking": true, "talkingTo": [10010], "contents": [{"sender": 10010, "target": 10006, "time": 1733814348917, "content": "Satoshi, this \u2018Quantum Brew\u2019 might just be the blend to unlock your next crypto idea\u2014let\u2019s see how it spins the taste of innovation!"}]}}]}

"""

In [3]:
test_json = json.loads(java_json)
# if "mapObj" in test_json:
#     del test_json["mapObj"]
test_json

{'world': {'time': 1878522183069},
 'npcs': [{'npcId': 10006,
   'status': 'free',
   'selling': [],
   'items': [],
   'action': {'param': {'gridX': 0.0, 'gridY': 0.0, 'npcId': 10010.0}},
   'curAction': {},
   'mapData': [],
   'surroundings': {'people': [{'npcId': 10007, 'status': 'free'},
     {'npcId': 10008, 'status': 'free'},
     {'npcId': 10009, 'status': 'free'},
     {'npcId': 10010, 'status': 'move'}],
    'items': []},
   'talk': {'isTalking': True,
    'talkingTo': [10010],
    'contents': [{'sender': 10010,
      'target': 10006,
      'time': 1733814348917,
      'content': 'Satoshi, this ‘Quantum Brew’ might just be the blend to unlock your next crypto idea—let’s see how it spins the taste of innovation!'}]}}]}

In [4]:
talkingInfo, is_talking = BhrLgcManualProcess.parse_talking_from_java(java_json)
is_idling = BhrLgcManualProcess.parse_isIdling(java_json)
    

inputInHumanString = BhrLgcManualProcess.parse_npc_info_for_nextaction(java_json)

is_findingToTalk = BhrLgcManualProcess.parse_isFindingPeopletoTalk(java_json) # Last action is finding people to talk
is_idling = BhrLgcManualProcess.parse_isIdling(java_json)
is_talking = BhrLgcManualProcess.parse_isTalking(java_json) 
is_buying = BhrLgcManualProcess.parse_isBuying(java_json)

target_sleeping, sleep_target_name = BhrLgcManualProcess.parse_target_sleeping(java_json) 
target_talking, talk_target_name = BhrLgcManualProcess.parse_target_talking(java_json)
shop_target_present, shopowner_target_name  = BhrLgcManualProcess.parse_target_oid_owner_at_shop(java_json)


to_memory = BhrLgcManualProcess.parse_npc_info_formemory(java_json)

print("talkingInfo:", talkingInfo)
print("is_talking:", is_talking)
print("is_idling:", is_idling)
print("inputInHumanString:", inputInHumanString)
print("is_findingToTalk:", is_findingToTalk)
print("is_talking:", is_talking)
print("is_buying:", is_buying)
print("target_sleeping:", target_sleeping)
print("sleep_target_name:", sleep_target_name)
print("target_talking:", target_talking)
print("talk_target_name:", talk_target_name)
print("shop_target_present:", shop_target_present)
print("shopowner_target_name:", shopowner_target_name)
print("to_memory:", to_memory)

I am free free
I am free free
talkingInfo: Pippin said to Satoshi: Satoshi, this ‘Quantum Brew’ might just be the blend to unlock your next crypto idea—let’s see how it spins the taste of innovation! 

is_talking: True
is_idling: True
inputInHumanString: 
    Now is 2029-07-12 11:43:03. 
    Pippin said to Satoshi: Satoshi, this ‘Quantum Brew’ might just be the blend to unlock your next crypto idea—let’s see how it spins the taste of innovation! 

    
is_findingToTalk: False
is_talking: True
is_buying: False
target_sleeping: False
sleep_target_name: Pippin
target_talking: False
talk_target_name: Pippin
shop_target_present: False
shopowner_target_name: None
to_memory: At 2029-07-12 11:43:03, Pippin said to Satoshi: Satoshi, this ‘Quantum Brew’ might just be the blend to unlock your next crypto idea—let’s see how it spins the taste of innovation! 



In [5]:
test_json

{'world': {'time': 1878522183069},
 'npcs': [{'npcId': 10006,
   'status': 'free',
   'selling': [],
   'items': [],
   'action': {'param': {'gridX': 0.0, 'gridY': 0.0, 'npcId': 10010.0}},
   'curAction': {},
   'mapData': [],
   'surroundings': {'people': [{'npcId': 10007, 'status': 'free'},
     {'npcId': 10008, 'status': 'free'},
     {'npcId': 10009, 'status': 'free'},
     {'npcId': 10010, 'status': 'move'}],
    'items': []},
   'talk': {'isTalking': True,
    'talkingTo': [10010],
    'contents': [{'sender': 10010,
      'target': 10006,
      'time': 1733814348917,
      'content': 'Satoshi, this ‘Quantum Brew’ might just be the blend to unlock your next crypto idea—let’s see how it spins the taste of innovation!'}]}}]}

In [6]:
import sys
import os
# Add the base directory (one level up from the current directory)
base_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
sys.path.append(base_dir)

from DBConnect import DBCon
from DBConnect import BhrDBJavaBuffer
from DBConnect import BhrDBInstruction
from DBConnect import BhrDBReflectionTracer
from DBConnect import BhrDBMemStre
from DBConnect import BhrDBReflection
from DBConnect import BhrDBSchedule

import BhrLgcGPTProcess
import BhrLgcManualProcess
import BhrLgcToMemStre

import pandas as pd
import numpy as np
import json
import re
import pickle
import hashlib
import configparser
import yaml

config = configparser.ConfigParser()
# Adjust path to look for config.ini in AImodule regardless of the current directory
base_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
config_path = os.path.join(base_dir, 'config.ini')
config.read(config_path)

yaml_path = os.path.join(base_dir, 'char_config.yaml')

# Load the YAML file

with open(yaml_path, 'r') as file:
    char_config = yaml.safe_load(file)
    print("YAML content loaded successfully.")


NameError: name '__file__' is not defined

In [ ]:

npc = next((npc for npc in char_config['npcCharacters'] if npc['npcId'] == npcId), None)
if not npc:
    raise ValueError(f"NPC with npcId {npcId} not found in char.yaml")
cur_schedule_str =  npc['schedule']